In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import pandas as pd
from pathlib import Path
from integrations import integration_base as base
from integrations import elc_integration as elc

# This is an optional step to make floating point numbers display more compactly.
# If you prefer other options, you can change or delete this without harm.
pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

In [2]:
# If you are beginning a *new* integration, or starting over

#base.integration_clean()
base.integration_start()

In [3]:
elc.setup(2018)

Integration year set to 2018
Previously defined grid mix loaded for year 2018


# Developer Section

In [32]:
import importlib
importlib.reload(elc)

<module 'integrations.elc_integration' from 'C:\\Working\\solutions\\integrations\\..\\integrations\\elc_integration.py'>

In [4]:
elc.step1_calculate_tams()

C:\Working\solutions\integrations\..\model\unitadoption.py:627: RuntimeWarning: invalid value encountered in subtract
  growth_array = np.max([growth.values[1:] - growth.values[:-1], np.full(growth.values[1:].shape, 0.)], axis=0)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\P

In [5]:
elc.step2_calculate_adoptions()

In [7]:
elc.es.pds_adoption['PDS1'].head(10)

Year,other biomass,coal,natural gas,oil products,large hydro,onshorewind,offshorewind,solarpvutil,solarpvroof,concentratedsolar,geothermal,waveandtidal,biomass,microwind,instreamhydro,nuclear,wastetoenergy,landfillmethane,biogas
2014,0.00802,0.44238,0.21878,0.04277,0.17314,0.03047,0.00109,0.005,0.00333,0.0004,0.00337,0.00004,0.00277,0.00005,0.02429,0.10719,0.00389,0.00106,0.00248
2015,0.00852,0.40619,0.23467,0.03817,0.16075,0.03559,0.00099,0.0068,0.00453,0.00036,0.00324,0.00009,0.00306,0.00007,0.02429,0.10696,0.00382,0.0012,0.00306
2016,0.00797,0.40075,0.23207,0.03664,0.16258,0.04119,0.00131,0.00916,0.0061,0.00032,0.00346,0.00012,0.00341,0.00008,0.02448,0.10725,0.00384,0.00129,0.00346
2017,0.00727,0.39583,0.22763,0.03528,0.15625,0.04667,0.00172,0.0115,0.00765,0.0004,0.00377,0.00014,0.00375,0.00009,0.02463,0.10771,0.00386,0.00138,0.00383
2018,0.00648,0.39277,0.22731,0.03413,0.15953,0.05204,0.00237,0.01384,0.00921,0.00059,0.00416,0.00014,0.0041,0.0001,0.02593,0.10831,0.00389,0.00145,0.0042
2019,0.00648,0.31641,0.24707,0.03048,0.16,0.0573,0.0028,0.01622,0.0108,0.00088,0.00463,0.00014,0.00443,0.00011,0.02483,0.10899,0.00391,0.00152,0.00454
2020,0.00648,0.29659,0.25689,0.02812,0.15954,0.0625,0.00344,0.01868,0.01244,0.00128,0.00516,0.00014,0.00476,0.00012,0.02489,0.10979,0.00394,0.00159,0.00488
2021,0.00648,0.27798,0.26476,0.0259,0.15912,0.06769,0.00416,0.02127,0.01416,0.00178,0.00575,0.00014,0.0051,0.00013,0.02494,0.11078,0.00397,0.00165,0.00522
2022,0.00648,0.26002,0.27205,0.02377,0.15847,0.07276,0.00494,0.02395,0.01595,0.00237,0.00639,0.00014,0.00542,0.00014,0.02495,0.11172,0.004,0.0017,0.00554
2023,0.00648,0.24289,0.27808,0.02173,0.15774,0.07778,0.00577,0.02679,0.01784,0.00307,0.00708,0.00014,0.00574,0.00015,0.02494,0.11272,0.00403,0.00176,0.00587


In [8]:
elc.es.pds_adoption['PDS1'].sum(axis=1)

2014   1.07053
2015   1.04237
2016   1.04549
2017   1.03937
2018   1.05055
2019   1.00156
2020   1.00124
2021   1.00097
2022   1.00077
2023   1.00059
2024   1.00043
2025   0.99988
2026   1.00026
2027   1.00018
2028   1.00012
2029   1.00007
2030   0.99966
2031   1.00022
2032   1.00042
2033   1.00063
2034   1.00087
2035   1.00121
2036   1.00138
2037   1.00161
2038   1.00181
2039   1.00197
2040   1.00276
2041   1.00197
2042   1.00146
2043   1.00076
2044   0.99992
2045   0.99947
2046   0.99778
2047   0.99646
2048   0.99506
2049   0.99349
2050   0.99289
2051   0.99352
2052   0.99357
2053   0.99372
2054   0.99405
2055   0.99455
2056   1.02098
2057    1.0487
2058   1.07648
2059   1.10466
2060   1.13284
dtype: float64